<a href="https://colab.research.google.com/github/adidam/rag-impl/blob/main/rag_visualizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install rank_bm25
!pip install datasets
!pip install gradio
!pip install pymilvus pymilvus[model]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 whi

In [2]:
import numpy as np
import os
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import random_split
import torch.optim as optim
import matplotlib.pyplot as plt
import nltk

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# **Tokenization and other utility functions**

In [3]:
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer

# embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = "BAAI/LLM-Embedder"
# embedding_model = "BAAI/bge-large-en"

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(embedding_model)
embedder = SentenceTransformer(embedding_model)  # Pretrained sentence transformer

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/28.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# New code - 12/4 10 pm
import hashlib
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import scipy.sparse as sp

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(embedding_model)
#tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Sliding window configuration
TOKEN_LIMIT = 512
SLIDING_WINDOW_OVERLAP = 100  # Overlap between consecutive chunks (in tokens)

# Function for chunking with token limit and sliding window
def chunk_with_token_limit(text, token_limit=512, overlap=100):
    sentences = sent_tokenize(text)  # Split text into sentences
    chunks = []  # Store resulting chunks
    current_chunk = []  # Temporarily hold sentences for the current chunk
    current_chunk_tokens = 0  # Token count for the current chunk

    for sentence in sentences:
        # Tokenize the sentence and calculate its token count
        sentence_tokens = tokenizer.tokenize(sentence)
        num_tokens = len(sentence_tokens)

        # print(f"Tokens: {sentence_tokens[0]}")

        # If adding this sentence exceeds the token limit
        if current_chunk_tokens + num_tokens > token_limit:
            # Save the current chunk
            chunk_text = " ".join(current_chunk)
            chunks.append(chunk_text)

            # Prepare the next chunk with overlap
            overlap_tokens = tokenizer.tokenize(" ".join(current_chunk[-1:]))
            current_chunk = [sentence for sentence in current_chunk[-(overlap // len(overlap_tokens)) :]] if current_chunk else []
            current_chunk_tokens = sum(len(tokenizer.tokenize(sent)) for sent in current_chunk)

        # Add the sentence to the current chunk
        current_chunk.append(sentence)
        current_chunk_tokens += num_tokens

    # Add the last chunk if it exists
    if current_chunk:
        chunk_text = " ".join(current_chunk)
        chunks.append(chunk_text)

    return chunks

def process_document_with_identifiers(document):
    processed_data = []
    title_count = -1  # to start from 0
    # print("document>>>>>>>",document)
    for section in document:
        section_chunks = []
        passage_count = [ord('a')]  # Passage identifier as a list to handle nested increments
        title_count += 1  # Increment title count

        # Tokenize the section into sentences
        sentences = sent_tokenize(section)
        for sentence in sentences:
            if sentence.startswith("Title:"):
                # New document detected
                identifier = f"{title_count}{''.join(chr(c) for c in passage_count)}"  # Identifier for the title

                # Commented this line to integrate and test with small to big. To be uncommented after testing
                chunked_texts = chunk_with_token_limit(sentence, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)

                #Added this line to replace sliding window tokenization with hybrid tokenization (sliding window + small-to-big)
                #chunked_texts = hybrid_chunking(sentence,tokenizer, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)

                for chunk in chunked_texts:
                    section_chunks.append([identifier, chunk])
                passage_count = [ord('a')]  # Reset passage count for the new document
            else:
                # Sentence under the current document
                identifier = f"{title_count}{''.join(chr(c) for c in passage_count)}"

                # Commented this line to integrate and test with small to big. To be uncommented after testing
                chunked_texts = chunk_with_token_limit(sentence, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)

                #Added this line to replace sliding window tokenization with hybrid tokenization (sliding window + small-to-big)
                #chunked_texts = hybrid_chunking(sentence,tokenizer, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)
                print("chunked_texts>>>>process_document_with_identifiers>>>>> "+ "".join(chunked_texts))

                for chunk in chunked_texts:
                    section_chunks.append([identifier, chunk])

                # Increment passage_count intelligently
                i = len(passage_count) - 1
                while i >= 0:
                    passage_count[i] += 1
                    if passage_count[i] > ord('z'):
                        passage_count[i] = ord('a')
                        if i == 0:
                            passage_count.insert(0, ord('a'))  # Add a new character to the identifier
                        i -= 1
                    else:
                        break


        print("section_chunks>>>>>>>",section_chunks)
        processed_data.append(section_chunks)

    return processed_data

# Function to generate a hash based on content and key metadata
def generate_hash(content, metadata):
    """Generate a unique hash for the document content and key metadata."""
    key_fields = f"{content}|{metadata.get('item_index')}|{metadata.get('prefix')}"
    return hashlib.md5(key_fields.encode('utf-8')).hexdigest()

# Function to retrieve existing hashes from the database
def get_existing_hashes(collection):
    """Retrieve all existing hashes (IDs) currently in the database."""
    all_records = collection.get(include=["documents", "metadatas"])  # Fetch documents and metadata
    existing_hashes = set()
    for doc, metadata in zip(all_records["documents"], all_records["metadatas"]):
        doc_hash = generate_hash(doc, metadata)
        existing_hashes.add(doc_hash)
    return existing_hashes

# Function to retrieve existing hashes from the database
def get_existing_hashes_milvus(all_records):
    """Retrieve all existing hashes (IDs) currently in the database."""
    existing_hashes = set()
    print(f"all records >>> {len(all_records)}")
    if all_records == None or len(all_records) == 0:
        return existing_hashes

    for record in all_records:
        doc = record.get("documents")
        metadata = record.get("metadata")
        doc_hash = generate_hash(doc, metadata)
        existing_hashes.add(doc_hash)
    return existing_hashes


# Tokenize corpus and prepare BM25 encoder
def prepare_bm25_encoder(texts):
    tokenized_corpus = [word_tokenize(text.lower()) for text in texts]
    bm25_encoder = BM25Okapi(tokenized_corpus)
    return bm25_encoder

def generate_sparse_vector_bm25(query, bm25_encoder):
    tokenized_query = word_tokenize(query.lower())
    scores = bm25_encoder.get_scores(tokenized_query)
    # Convert scores to CSR format
    sparse_vector = sp.csr_matrix(scores)
    return sparse_vector

# **Milvus VectorDataStore class**

In [8]:
import numpy as np
import time
from pymilvus import connections
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
from pymilvus import MilvusClient
from pymilvus import utility

class VectorDataStore:
    db_url = "http://localhost:19530"

    #description = f"collection created for {self.name}"

    def __init__(self, path="/content/ragbench.db"):
        self.client = MilvusClient(path)


    def get_or_create_collection(self, name, vec_dim=128):
        try:
            self.get_collection(name)
        except:
            print(f"Collection {name} doesn't exist. Creating...")
            self.create_collection(name, vec_dim)


    def create_collection(self, name, vec_dim=128):
        if self.client.has_collection(name):
            self.default_collection_name = name

        self.description = f"collection to store {name}"

        index_params = self.client.prepare_index_params()
        index_params.add_index(
            field_name="embedding",
            index_type="AUTOINDEX",
            metric_type="COSINE"
        )
        index_params.add_index(
            field_name="sparse",
            index_type="SPARSE_INVERTED_INDEX",
            metric_type="IP"
        )
        schema = self.client.create_schema(
            auto_id=False,
            enable_dynamic_fields=True,
        )
        schema.add_field(field_name="pk", datatype=DataType.VARCHAR, max_length=64, is_primary=True)
        schema.add_field(field_name="metadata", datatype=DataType.JSON)
        schema.add_field(field_name="documents", datatype=DataType.VARCHAR, max_length=512)
        schema.add_field(field_name="sparse", datatype=DataType.SPARSE_FLOAT_VECTOR)
        schema.add_field(field_name="embedding", datatype=DataType.FLOAT_VECTOR, dim=vec_dim)

        collection = self.client.create_collection(collection_name=name,
                                       schema=schema,
                                       index_params=index_params)
        self.current_collection = collection
        return collection


    def get_collection(self, name):
        if not self.client.has_collection(name):
            raise ValueError(f"Collection '{name}' does not exist.")
        self.current_collection = Collection(name)
        return self.current_collection

    def get_all_records(self, collection):
        all_records = self.client.query(
            collection_name=collection,
            filter=None,
            output_fields=["documents", "metadata"],
            limit=10000
        )
        if all_records == None:
            all_records = []

        return all_records

    def has_entities(self, name):
        if not self.client.has_collection(name):
            raise ValueError(f"Collection '{name}' does not exists.")
        self.default_collection = name
        collection_stats = self.client.get_collection_stats(collection_name)
        count = collection_stats.get("row_count", 0)  # Retrieve the number of entities
        return count

    def insert(self, collection_name: str, metadata: list[dict[str, any]],
                documents: list[str], sparse_embs: np.ndarray, embeddings: np.ndarray, ids: list[int]):

        if not self.client.has_collection(collection_name):
            raise ValueError(f"Collection '{collection_name}' does not exist. Create it first.")

        if len(metadata) != len(embeddings) != len(documents) != len(ids):
           raise ValueError("Metadata, documnets, ids and embeddings must have the same length.")

        data = []
        for meta, doc, sp_embs, emb, id in zip(metadata, documents, sparse_embs, embeddings, ids):
          datum = {
              "pk": id,
              "metadata": meta,
              "documents": doc,
              "sparse": sp_embs,
              "embedding": emb.tolist(),
          }
          data.append(datum)

        self.client.insert(collection_name, data)
        print(f"Inserted {len(metadata)} records into collection '{collection_name}'.")

    def drop_collection(self, collection_name: str):
        if not self.client.has_collection(collection_name):
            raise ValueError(f"Collection '{collection_name}' does not exist.")
        self.client.drop_collection(collection_name)
        print(f"Dropped collection '{collection_name}'.")

    def delete_all(self, collection_name: str):
        if not self.client.has_collection(collection_name):
            raise ValueError(f"Collection '{collection_name}' does not exist.")
        self.client.delete(collection_name, expr="pk >= 0")
        self.client.flush([collection_name])

    def search(self, query_embedding: np.ndarray, collection='all', top_k: int = 10) -> list[dict[str, any]]:
        """
        Search across all collections for the top-k closest embeddings.
        :param query_embedding: The embedding vector to search for.
        :param top_k: Number of top results to retrieve.
        :return: A list of dictionaries containing collection name, id, metadata, and distance.
        """
        results = []
        if collection=='all':
          collections = self.client.list_collections()
        else:
          collections = [collection]

        start_time = time.time()
        for collection_name in collections:
            if not self.client.has_collection(collection_name):
                continue

            # Set params to COSINE to match chromadb
            search_params = {"metric_type": "COSINE", "params": {"ef": 128}}

            search_results = self.client.search(
                collection_name=collection_name,
                data=[query_embedding],
                anns_field="embedding",
                search_params=search_params,
                limit=top_k,
                output_fields=["metadata", "documents"]
            )

            print(f"search results size : {len(search_results)}")

            for hits in search_results:
                for hit in hits:
                    print(f"Collection: {collection_name}, data: {str(hit)}")
                    results.append({
                        "collection": collection_name,
                        "id": hit["id"],
                        "metadata": hit["entity"]["metadata"],
                        "distance": hit["distance"],
                        "documents": hit["entity"]["documents"]
                      })

        results = sorted(results, key=lambda x: x["distance"])[:top_k]
        end_time = time.time()
        print(f"Search completed. Found {len(results)} results. in {end_time - start_time} secs")
        return results

    def sparse_search(self, query_embedding: np.ndarray, collection='all', top_k : int=10)-> list[dict[str, any]] :
        results = []
        if (collection=='all'):
          collections = self.client.list_collections()
        else:
          collections = [collection]
        start_time = time.time()
        for collection_name in collections:
            if not self.client.has_collection(collection_name):
                continue

            # Set params to COSINE to match chromadb
            search_params = {"metric_type": "IP", "params": {"ef": 128}}

            search_results = self.client.search(
                collection_name=collection_name,
                data=[query_embedding],
                anns_field="sparse",
                search_params=search_params,
                limit=top_k,
                output_fields=["metadata", "documents"]
            )

            print(f"search results size : {len(search_results)}")

            for hits in search_results:
                for hit in hits:
                    print(f"Collection: {collection_name}, data: {str(hit)}")
                    results.append({
                        "collection": collection_name,
                        "id": hit["id"],
                        "metadata": hit["entity"]["metadata"],
                        "distance": hit["distance"],
                        "documents": hit["entity"]["documents"]
                      })

        results = sorted(results, key=lambda x: x["distance"])[:top_k]
        end_time = time.time()
        print(f"Search completed. Found {len(results)} results. in {end_time - start_time} secs")
        return results

    def hybrid_search(self, sparse_query_embedding: np.ndarray, dense_query_embedding: np.ndarray, collection='all', top_k : int=10, alpha=0.3)-> list[dict[str, any]] :
        results = []
        start_time = time.time()
        sparse_results = self.sparse_search(sparse_query_embedding, collection, top_k)
        n = int(len(sparse_results) * alpha)
        alpha_sparse_results = sparse_results[:n]
        dense_results = self.search(dense_query_embedding, collection, top_k)
        #'results = sorted(results, key=lambda x: x["distance"])[:top_k]
        results = dense_results + alpha_sparse_results
        end_time = time.time()
        print(f"Hybrid Search completed. Found {len(results)} results. in {end_time - start_time} secs")
        return results

    def extract_documents(self, search_results: list[dict[str, any]]) -> list[np.ndarray]:
      """
      Extract embedding values from search results.
      :param search_results: List of dictionaries containing search results.
      :return: List of embedding vectors as NumPy arrays.
      """
      return [result["documents"] for result in search_results if "documents" in result]

In [12]:
datasets = ['covidqa','cuad','delucionqa','emanual','expertqa','finqa','hagrid','hotpotqa','msmarco','pubmedqa','tatqa','techqa']
datastor = VectorDataStore()

insert_data = True
store_client = "Milvus"
num_records = 0

vector_dim = embedder.get_sentence_embedding_dimension()

In [ ]:
g_data_index = random.randint(1, 12) - 1
dataset = datasets[g_data_index]
print(f"Selected dataset: {datasets[g_data_index]}")
milvus_collection_name = f"ragbench_collection_{dataset}_v16"
datastor.drop_collection(milvus_collection_name)

data = load_dataset("rungalileo/ragbench", datasets[g_data_index], split="train")
top_5_rows = data.select(range(5))

In [15]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import random

# Initialize storage for documents, IDs, and metadata

for i in range(11):
  all_documents = []
  all_ids = []
  all_metadatas = []

  # Process each dataset
  doc_idx = 0  # Global document index for unique IDs
  for dataset in [datasets[i]]:
      milvus_collection_name = f"ragbench_collection_{dataset}_v16"
      try:
        num_records = datastor.has_entities(milvus_collection_name)
      except:
        num_records = 0
        print("we are good. collection doesnt exists")

      if num_records > 0:
          datastor.drop_collection(milvus_collection_name)
      datastor.get_or_create_collection(milvus_collection_name, vector_dim)

      print(f"Processing {dataset}...")
      data = load_dataset("rungalileo/ragbench", dataset, split="train")
      #only select first 5 records for debugging duplicate records. **PLEASE REMOVE THIS AFTER DEBUGGING**
      data = data.select(range(10))
      for idx, row in tqdm(enumerate(data), desc=f"Processing {dataset}"):
          # Extract document text
          print(f"~~~~~~>>>  question: {row.get('question', '')}")
          doc_text = row.get('documents', '')

          # Skip if no documents found
          if not doc_text:
              continue

          # Process the document
          processed_output = process_document_with_identifiers(doc_text)
          added_item_idxs = set()

          # Populate the lists
          for section_idx, section in enumerate(processed_output):
              for item_idx, (prefix, content) in enumerate(section):
                  # Skip if this item_idx has already been processed
                  if item_idx in added_item_idxs:
                      continue

                  # Add the item_idx to the set to track it
                  added_item_idxs.add(item_idx)

                  # Add the document
                  document = f"[{prefix}] {content}"
                  all_documents.append(document)

                  # Construct a globally unique ID
                  doc_id = f"{dataset}_{doc_idx}_{section_idx}_{item_idx}"
                  all_ids.append(doc_id)

                  # Construct metadata
                  metadata = {
                      "dataset": dataset,
                      "global_index": doc_idx,
                      "section_index": section_idx,
                      "item_index": item_idx,
                      "prefix": prefix,
                      "type": "Title" if prefix.endswith("a") else "Passage",
                  }
                  all_metadatas.append(metadata)

          doc_idx += 1  # Increment global document index

  # Step 4: Generate Embeddings
  embedder = SentenceTransformer(embedding_model)  # Pretrained sentence transformer
  batch_size = 2500  # Adjust based on available memory

  # Generate embeddings in batches
  all_embeddings = []
  for i in tqdm(range(0, len(all_documents), batch_size), desc="Generating embeddings"):
      batch_docs = all_documents[i:i + batch_size]
      batch_embeddings = embedder.encode(batch_docs, show_progress_bar=True)
      all_embeddings.extend(batch_embeddings)

  bm25_encoder = prepare_bm25_encoder(all_documents)
  sparse_vectors = [generate_sparse_vector_bm25(text, bm25_encoder) for text in all_documents]

  # Adding data to milvus with enhanced duplicate check
  all_recs = datastor.get_all_records(milvus_collection_name)
  #print(f"sample: {str(all_recs[0])}")
  existing_hashes = get_existing_hashes_milvus(all_recs)

  for i in tqdm(range(0, len(all_documents), batch_size), desc="Adding data to DB"):
      batch_embeddings = all_embeddings[i:i + batch_size]
      batch_sparse_embs = sparse_vectors[i:i + batch_size]
      batch_metadatas = all_metadatas[i:i + batch_size]
      batch_documents = all_documents[i:i + batch_size]
      batch_ids = []

      # Generate hashes for each document in the batch
      for doc, metadata in zip(batch_documents, batch_metadatas):
          doc_hash = generate_hash(doc, metadata)
          if doc_hash not in existing_hashes:
              batch_ids.append(doc_hash)
              existing_hashes.add(doc_hash)  # Add hash to local set to avoid duplicates in the same batch
          else:
              print(f"Skipping duplicate document: {doc[:15]}...")  # Print a preview of the duplicate doc

      # Add non-duplicate documents to the database
      if batch_ids:  # Ensure there are non-duplicate documents to add
          # Add the batch to the Milvus collection
          if store_client == "Milvus" and insert_data:
              datastor.insert(milvus_collection_name,
                  metadata=batch_metadatas,
                  documents=batch_documents,
                  sparse_embs = np.array(batch_sparse_embs),
                  embeddings=np.array(batch_embeddings),
                  ids=batch_ids
              )

  print(f"total data inserted into {milvus_collection_name} iteration {i}")
  print(f"records : {datastor.has_entities(milvus_collection_name)} ")

we are good. collection doesnt exists
Collection ragbench_collection_covidqa_v16 doesn't exist. Creating...
Processing covidqa...


Processing covidqa: 10it [00:00, 53.79it/s]


~~~~~~>>>  question: What role does T-cell count play in severe human adenovirus type 55 (HAdV-55) infection?
chunked_texts>>>>process_document_with_identifiers>>>>> Chen et al.
chunked_texts>>>>process_document_with_identifiers>>>>> reported that, in the acute phase of HAdV-55 infection, patients with severe disease may have high levels of dendritic cells and Th17 cells .
chunked_texts>>>>process_document_with_identifiers>>>>> In our study, the only patient who recovered from severe infection had higher T-cell counts.
chunked_texts>>>>process_document_with_identifiers>>>>> Three of the five patients had relatively low T-cell counts when admitted.
chunked_texts>>>>process_document_with_identifiers>>>>> Our results suggest that these three patients may have been relatively immunocompromised and that a lower T-cell count may be a risk
section_chunks>>>>>>> [['0a', 'Title: Emergent severe acute respiratory distress syndrome caused by adenovirus type 55 in immunocompetent adults in 2013: a

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 1/1 [00:11<00:00, 11.18s/it]


all records >>> 57


Adding data to DB: 100%|██████████| 1/1 [00:00<00:00, 514.89it/s]


Skipping duplicate document: [0a] Title: Eme...
Skipping duplicate document: [0a] Chen et al...
Skipping duplicate document: [0b] reported t...
Skipping duplicate document: [0c] In our stu...
Skipping duplicate document: [0d] Three of t...
Skipping duplicate document: [0e] Our result...
Skipping duplicate document: [0a] Title: Gen...
Skipping duplicate document: [0a] The Morpho...
Skipping duplicate document: [1b] Viral appl...
Skipping duplicate document: [2c] Antigen-sp...
Skipping duplicate document: [2d] To...
Skipping duplicate document: [0a] Title: MER...
Skipping duplicate document: [0a] These reli...
Skipping duplicate document: [0b] No sign of...
Skipping duplicate document: [0c] Nor did IF...
Skipping duplicate document: [0d] Of 226 sla...
Skipping duplicate document: [0a] Title: Kin...
Skipping duplicate document: [0a] The cells ...
Skipping duplicate document: [0b] The tumors...
Skipping duplicate document: [0c] The cells ...
Skipping duplicate document: [0d] The live a...


NameError: name 'collection_name' is not defined

In [ ]:
import gradio as gr

# Placeholder function to get response from your LLM/RAG pipeline
def get_response(query, model_name):
    response = f"Response from {model_name} for query: {query}"  # Replace with actual model call
    return response

# Available LLM models
llm_models = ["Llama-3.2-1B", "GPT-4", "Mistral", "Custom-LLM"]  # Modify as needed

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## LLM/RAG Query Interface")

    with gr.Row():
        query_input = gr.Textbox(label="Enter your query")
        model_selector = gr.Dropdown(llm_models, label="Select LLM Model")

    submit_btn = gr.Button("Submit")
    response_output = gr.Textbox(label="Response", interactive=False)

    submit_btn.click(get_response, inputs=[query_input, model_selector], outputs=response_output)

# Launch the interface
demo.launch()
